# Elastic Bounds: Voigt–Reuss–Hill and Hashin–Shtrikman

This tutorial reproduces the elastic modulus examples using the package functions from `drp_template.compute.rockphysics.bounds` and the plotting helper from `drp_template.image`.

We compute Voigt, Reuss, Hill (VRH) and Hashin–Shtrikman (HS) bounds for two-phase mixtures across a volume fraction range and visualize the results.

In [ ]:
# Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
from drp_template.compute.rockphysics.bounds import voigt_reuss_hill_bounds, hashin_shtrikman_bounds
from drp_template.image import plot_effective_modulus
from drp_template.compute.conversions import Pa2GPa

# Reload modules to ensure we have the latest changes
import importlib
import drp_template.compute.rockphysics.bounds.voigt_reuss
import drp_template.compute.rockphysics.bounds.hashin_shtrikman
from drp_template.compute.rockphysics.bounds import voigt_reuss_hill_bounds, hashin_shtrikman_bounds

In [ ]:
# Material properties (Pa)
# Quartz
bulk_modulus_quartz = 37e9
shear_modulus_quartz = 44e9

# Clay
bulk_modulus_clay = 21e9
shear_modulus_clay = 7e9

# Water (bulk only, shear ~0)
bulk_modulus_water = 2.4e9
shear_modulus_water = 0.0

materials_bulk = np.array([bulk_modulus_quartz, bulk_modulus_clay])
materials_shear = np.array([shear_modulus_quartz, shear_modulus_clay])

phi = np.linspace(0, 1, 100)  # Volume fraction of Clay (second constituent)

# Storage arrays
bulk_modulus_voigt_arr = np.zeros_like(phi)
bulk_modulus_reuss_arr = np.zeros_like(phi)
bulk_modulus_hill_arr = np.zeros_like(phi)
shear_modulus_voigt_arr = np.zeros_like(phi)
shear_modulus_reuss_arr = np.zeros_like(phi)
shear_modulus_hill_arr = np.zeros_like(phi)

bulk_modulus_hs_upper_arr = np.zeros_like(phi)
bulk_modulus_hs_lower_arr = np.zeros_like(phi)
shear_modulus_hs_upper_arr = np.zeros_like(phi)
shear_modulus_hs_lower_arr = np.zeros_like(phi)

for i, v_clay in enumerate(phi):
    v_quartz = 1.0 - v_clay
    fractions = [v_quartz, v_clay]

    vrh = voigt_reuss_hill_bounds(fractions, materials_bulk, materials_shear)
    hs = hashin_shtrikman_bounds(fractions, materials_bulk, materials_shear)

    bulk_modulus_voigt_arr[i] = vrh['bulk_modulus_voigt']
    bulk_modulus_reuss_arr[i] = vrh['bulk_modulus_reuss']
    bulk_modulus_hill_arr[i] = vrh['bulk_modulus_hill']
    shear_modulus_voigt_arr[i] = vrh['shear_modulus_voigt']
    shear_modulus_reuss_arr[i] = vrh['shear_modulus_reuss']
    shear_modulus_hill_arr[i] = vrh['shear_modulus_hill']

    bulk_modulus_hs_upper_arr[i] = hs['bulk_modulus_upper']
    bulk_modulus_hs_lower_arr[i] = hs['bulk_modulus_lower']
    shear_modulus_hs_upper_arr[i] = hs['shear_modulus_upper']
    shear_modulus_hs_lower_arr[i] = hs['shear_modulus_lower']

print("\nBounds at extremes:")
print(f"100% Quartz - Bulk: {Pa2GPa(bulk_modulus_voigt_arr[0]):.3f} GPa, Shear: {Pa2GPa(shear_modulus_voigt_arr[0]):.3f} GPa")
print(f"100% Clay   - Bulk: {Pa2GPa(bulk_modulus_voigt_arr[-1]):.3f} GPa, Shear: {Pa2GPa(shear_modulus_voigt_arr[-1]):.3f} GPa")

In [ ]:
# Plot: Effective Bulk Modulus across volume fraction
bulk_moduli_data = {
    'voigt': Pa2GPa(bulk_modulus_voigt_arr),
    'reuss': Pa2GPa(bulk_modulus_reuss_arr),
    'avg': Pa2GPa(bulk_modulus_hill_arr),
    'hs_upper': Pa2GPa(bulk_modulus_hs_upper_arr),
    'hs_lower': Pa2GPa(bulk_modulus_hs_lower_arr),
}

fig, ax = plot_effective_modulus(
    fraction=phi,
    data=bulk_moduli_data,
    title='Effective Bulk Modulus',
    marker=None,
    types=['voigt', 'reuss', 'avg', 'hs_upper', 'hs_lower'],
    dark_mode=False,
    linewidth=2,
    ylabel=('Quartz (GPa)', 'Clay (GPa)'),
    xlabel='Volume fraction of Clay',
)
plt.show()
fig.savefig("bulk_modulus_hs_bounds.png", format='png', dpi=300)

In [ ]:
# Single volume fraction example (Clay = 50%)
phi_point = 0.5
fractions_point = [1-phi_point, phi_point]
vrh_point = voigt_reuss_hill_bounds(fractions_point, materials_bulk, materials_shear)
hs_point = hashin_shtrikman_bounds(fractions_point, materials_bulk, materials_shear)

print(f"Effective moduli for {phi_point*100:.1f}% Clay (GPa):")
print(f"Bulk (Voigt): {Pa2GPa(vrh_point['bulk_modulus_voigt']):.3f}")
print(f"Bulk (Reuss): {Pa2GPa(vrh_point['bulk_modulus_reuss']):.3f}")
print(f"Bulk (Hill):  {Pa2GPa(vrh_point['bulk_modulus_hill']):.3f}")
print(f"Bulk (HS upper): {Pa2GPa(hs_point['bulk_modulus_upper']):.3f}")
print(f"Bulk (HS lower): {Pa2GPa(hs_point['bulk_modulus_lower']):.3f}")

point_bulk_data = {
    'voigt': [Pa2GPa(vrh_point['bulk_modulus_voigt'])],
    'reuss': [Pa2GPa(vrh_point['bulk_modulus_reuss'])],
    'avg': [Pa2GPa(vrh_point['bulk_modulus_hill'])],
    'hs_upper': [Pa2GPa(hs_point['bulk_modulus_upper'])],
    'hs_lower': [Pa2GPa(hs_point['bulk_modulus_lower'])],
}

fig, ax = plot_effective_modulus(
    fraction=phi_point,
    data=point_bulk_data,
    title='Effective Bulk Modulus (Single Point)',
    marker='o',
    markersize=10,
    types=['voigt', 'reuss', 'avg', 'hs_upper', 'hs_lower'],
    linewidth=2,
    ylabel=('Quartz (GPa)', 'Clay (GPa)'),
    xlabel='Volume fraction of Clay',
    loc_legend='best',
    ylim_off=1,
    xlim_off=0.25,
)
plt.show()
fig.savefig("effective_bulk_modulus_point.png", format='png', dpi=300)

In [ ]:
# Overlay example: full curve + single point (Clay = 60%)
phi_overlay = 0.6
fractions_overlay = [1-phi_overlay, phi_overlay]
vrh_overlay = voigt_reuss_hill_bounds(fractions_overlay, materials_bulk, materials_shear)

bulk_moduli_data_full = {
    'voigt': Pa2GPa(bulk_modulus_voigt_arr),
    'reuss': Pa2GPa(bulk_modulus_reuss_arr),
    'avg': Pa2GPa(bulk_modulus_hill_arr),
    'hs_upper': Pa2GPa(bulk_modulus_hs_upper_arr),
    'hs_lower': Pa2GPa(bulk_modulus_hs_lower_arr),
}

fig, ax = plot_effective_modulus(
    fraction=phi,
    data=bulk_moduli_data_full,
    title='Effective Bulk Modulus with Highlighted Point',
    marker=None,
    types=['voigt', 'reuss', 'avg', 'hs_upper', 'hs_lower'],
    dark_mode=False,
    linewidth=2,
    ylabel=('Quartz (GPa)', 'Clay (GPa)'),
    xlabel='Volume fraction of Clay',
)

ax.plot([phi_overlay], [Pa2GPa(vrh_overlay['bulk_modulus_voigt'])], 'ro', markersize=10, label='Voigt at 60% Clay')
ax.plot([phi_overlay], [Pa2GPa(vrh_overlay['bulk_modulus_reuss'])], 'bo', markersize=10, label='Reuss at 60% Clay')
ax.legend()
plt.show()
fig.savefig("effective_bulk_modulus_merge.png", format='png', dpi=300)

In [ ]:
# Clay–Water mixture example: sweep and plot bulk modulus
# Define materials: Clay (1) and Water (2)
materials_bulk_cw = np.array([bulk_modulus_clay, bulk_modulus_water])
materials_shear_cw = np.array([shear_modulus_clay, shear_modulus_water])

phi_water = np.linspace(0, 1, 100)  # Volume fraction of Water (second constituent)

bulk_modulus_voigt_cw = np.zeros_like(phi_water)
bulk_modulus_reuss_cw = np.zeros_like(phi_water)
bulk_modulus_hill_cw = np.zeros_like(phi_water)
shear_modulus_voigt_cw = np.zeros_like(phi_water)
shear_modulus_reuss_cw = np.zeros_like(phi_water)
shear_modulus_hill_cw = np.zeros_like(phi_water)
bulk_modulus_hs_upper_cw = np.zeros_like(phi_water)
bulk_modulus_hs_lower_cw = np.zeros_like(phi_water)
shear_modulus_hs_upper_cw = np.zeros_like(phi_water)
shear_modulus_hs_lower_cw = np.zeros_like(phi_water)

for i, v_w in enumerate(phi_water):
    v_cl = 1.0 - v_w
    fractions_cw = [v_cl, v_w]

    vrh_cw = voigt_reuss_hill_bounds(fractions_cw, materials_bulk_cw, materials_shear_cw)
    hs_cw = hashin_shtrikman_bounds(fractions_cw, materials_bulk_cw, materials_shear_cw)

    bulk_modulus_voigt_cw[i] = vrh_cw['bulk_modulus_voigt']
    bulk_modulus_reuss_cw[i] = vrh_cw['bulk_modulus_reuss']
    bulk_modulus_hill_cw[i] = vrh_cw['bulk_modulus_hill']
    shear_modulus_voigt_cw[i] = vrh_cw['shear_modulus_voigt']
    shear_modulus_reuss_cw[i] = vrh_cw['shear_modulus_reuss']
    shear_modulus_hill_cw[i] = vrh_cw['shear_modulus_hill']

    bulk_modulus_hs_upper_cw[i] = hs_cw['bulk_modulus_upper']
    bulk_modulus_hs_lower_cw[i] = hs_cw['bulk_modulus_lower']
    shear_modulus_hs_upper_cw[i] = hs_cw['shear_modulus_upper']
    shear_modulus_hs_lower_cw[i] = hs_cw['shear_modulus_lower']



bulk_moduli_cw = {
    'voigt': Pa2GPa(bulk_modulus_voigt_cw),
    'reuss': Pa2GPa(bulk_modulus_reuss_cw),
    'avg': Pa2GPa(bulk_modulus_hill_cw),
    'hs_upper': Pa2GPa(bulk_modulus_hs_upper_cw),
    'hs_lower': Pa2GPa(bulk_modulus_hs_lower_cw),
}

fig, ax = plot_effective_modulus(
    fraction=phi_water,
    data=bulk_moduli_cw,
    title='Effective Bulk Modulus (Clay–Water)',
    marker=None,
    types=['voigt', 'reuss', 'avg', 'hs_upper', 'hs_lower'],
    dark_mode=False,
    linewidth=2,
    ylabel=('Clay (GPa)', 'Water (GPa)'),
    xlabel='Volume fraction of Water',
)
plt.show()
fig.savefig("bulk_modulus_clay_water.png", format='png', dpi=300)

# Shear modulus plot for Clay–Water

shear_moduli_cw = {
    'voigt': Pa2GPa(shear_modulus_voigt_cw),
    'reuss': Pa2GPa(shear_modulus_reuss_cw),
    'avg': Pa2GPa(shear_modulus_hill_cw),
    'hs_upper': Pa2GPa(shear_modulus_hs_upper_cw),
    'hs_lower': Pa2GPa(shear_modulus_hs_lower_cw),
}

fig, ax = plot_effective_modulus(
    fraction=phi_water,
    data=shear_moduli_cw,
    title='Effective Shear Modulus (Clay–Water)',
    marker=None,
    types=['voigt', 'reuss', 'avg', 'hs_upper', 'hs_lower'],
    dark_mode=False,
    linewidth=2,
    ylabel=('Clay (GPa)', 'Water (GPa)'),
    xlabel='Volume fraction of Water',
)
plt.show()
fig.savefig("shear_modulus_clay_water.png", format='png', dpi=300)